# Verified Z Gate
Here we exemplify how a verified CNOT can be implemented. First we construct the usual MBQC pattern for a Z gate. This is a three quibt graph state with the first quibt measured in a rotated basis.

In this case we also include an `vertex_is_dummy_list`. This list can be thought of as a colouring of the graph. In particular each sub list corresponds to a colour, with a vertex being dummy when the corresponding entry in the list is `True`. Each vertex should be coloured by one of the sub lists, which it to say is `False` atleast once.

In [ ]:
import networkx as nx
import pytest

from ocvqc_py import GraphCircuit

graph_circuit = GraphCircuit(
    n_physical_qubits=2,
    n_logical_qubits=3,
    vertex_is_dummy_list=[
        [True, False, True],
        [False, True, False],
    ],
)

vertex_one = graph_circuit.add_graph_vertex(measurement_order=0)
vertex_two = graph_circuit.add_graph_vertex(measurement_order=1)

graph_circuit.add_edge(vertex_one, vertex_two)
graph_circuit.corrected_measure(vertex=vertex_one, t_multiple=4)

vertex_three = graph_circuit.add_graph_vertex(measurement_order=None)
graph_circuit.add_edge(vertex_two, vertex_three)
graph_circuit.corrected_measure(vertex=vertex_two, t_multiple=0)

graph_circuit.corrected_measure(vertex=vertex_three, t_multiple=0)

nx.draw(graph_circuit.entanglement_graph, with_labels=True)

Below we run the circuit as usual.

In [ ]:
from pytket.extensions.quantinuum import QuantinuumAPIOffline, QuantinuumBackend

backend = QuantinuumBackend(
    device_name="H1-1LE",
    api_handler=QuantinuumAPIOffline(),
)

n_shots = 1000
compiled_circuit = backend.get_compiled_circuit(circuit=graph_circuit)
result = backend.run_circuit(circuit=compiled_circuit, n_shots=n_shots)

A word on what has been run. At each shot a random bit is generated by measuring a plus state. This bit indicates if this is a test or compute round. In the case of a compute round the the computation proceeds as normal and we can recover the ideal measurement outcomes. Note that the ideal outcome 1 appears for about 500 shots. This is every compute round.

In [ ]:
graph_circuit.get_output_result(result=result).get_counts()

In the case of test rounds a random colour is chosen and dummy and test qubits are generated accordingly. We can then check how many of the test vertices fail, which is to say are not measured as their input. For the ideal simulation conducted here this will be 0.

In [ ]:
graph_circuit.get_failure_rate(result=result)